# Working with the ScienceImage Class [v1.1]

In [1]:
# imports
import os
import numpy as np
from importlib import reload

In [2]:
# Path to PYPIT-Development-suite
pypdev_path = os.getenv('PYPIT_DEV')

----

# Development

In [3]:
from pypit.core import arprocimg
from pypit import biasframe
from pypit import flatfield
from pypit import processimages
from pypit import scienceimage
from pypit import traceslits
from pypit import waveimage
from pypit import wavetilts

## Settings

In [4]:
settings = dict(masters={})
settings['masters']['directory'] = pypdev_path+'/Cooked/MF_shane_kast_blue'
settings['masters']['reuse'] = True
settings['masters']['loaded'] = []
#
setup = 'A_01_aa'
#
settings['combine'] = {}

In [5]:
spectrograph = 'shane_kast_blue'
det = 1

In [6]:
settings['detector'] = {}
settings['detector']['num'] = det
settings['detector']['dataext'] = 0
settings['detector']['datasec01'] = [[0, 1024], [0, 0]]
settings['detector']['datasec02'] = [[1024, 2048], [0, 0]]
settings['detector']['oscansec01'] = [[2049, 2080], [0, 0]]
settings['detector']['oscansec02'] = [[2080, 2111], [0, 0]]
settings['detector']['naxis0'] = 2112  # Raw frame, with overscan
settings['detector']['naxis1'] = 350
settings['detector']['numamplifiers'] = 2
settings['detector']['gain'] = [1.2, 1.2]
settings['detector']['ronoise'] = [3.7, 3.7]
settings['detector']['saturation'] = 65535.
settings['detector']['nonlinear'] = 0.76
settings['detector']['dispaxis'] = 1
settings['detector']['darkcurr'] = 0.
settings['detector']['binning'] = '1x1'

In [7]:
settings['trace'] = {}
settings['trace']['object'] = {}
settings['trace']['object']['order'] = 2                # What is the order of the polynomial function to be used to fit the object trace in each slit
settings['trace']['object']['function'] = 'legendre'    # What function should be used to trace the object in each slit? (polynomial, legendre, chebyshev)
settings['trace']['object']['find'] = 'standard'         # What algorithm to use for finding objects [standard, nminima]
settings['trace']['object']['nsmooth'] = 3              # Parameter for Gaussian smoothing when the nminima algorithm is used
settings['trace']['object']['xedge'] = 0.03             # Ignore any objects within xedge of the edge of the slit

In [8]:
settings['science'] = {}
settings['science']['extraction'] = {}
settings['science']['extraction']['reuse'] = False          # If the science frame has previously been extracted and saved, load the extractions
settings['science']['extraction']['profile'] = 'gaussian'   # Fitting function used to extract science data, only if the extraction is 2D (options are: gaussian, gaussfunc, moffat, moffatfunc) ### NOTE: options with suffix 'func' fits a function to the pixels whereas those without this suffix takes into account the integrated function within each pixel (and is closer to truth)
settings['science']['extraction']['maxnumber'] = 999        # Maximum number of objects to extract in a science frame
settings['science']['extraction']['manual01'] = {}
settings['science']['extraction']['manual01']['frame'] = None
settings['science']['extraction']['manual01']['params'] = None # Info for desired extraction [det,x_pixel_location, y_pixel_location,[x_range,y_range]]


## Load up

In [9]:
# Bias
settings['bias'] = {}
settings['bias']['useframe'] = 'bias'
#
biasFrame = biasframe.BiasFrame(setup=setup, settings=settings)
msbias = biasFrame.master()
msbias.shape

[WARNING] :: processimages.py 110 __init__() - No information on the spectrograph was given.  Do not attempt to (re)process the images
[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2112, 350)

In [10]:
# Traceslits
settings['trace']['slits'] = traceslits.default_settings()['trace']['slits'].copy()
traceSlits = traceslits.TraceSlits(None, None, setup=setup, settings=settings)
traceSlits.master()

[INFO]    :: traceslits.py 522 _make_pixel_arrays() - Converting physical trace locations to nearest pixel
[INFO]    :: traceslits.py 529 _make_pixel_arrays() - Identifying the pixels belonging to each slit


True

In [11]:
# Wavelengths
waveImage = waveimage.WaveImage(setup=setup, settings=settings)
wave = waveImage.master()
wave.shape

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2048, 350)

In [12]:
# Tilts
waveTilts = wavetilts.WaveTilts(None, setup=setup, settings=settings)
tilts = waveTilts.master()
tilts.shape

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2048, 350)

In [13]:
# Flat
flatField = flatfield.FlatField(settings=settings, setup=setup)
msflat = flatField.master()

[WARNING] :: processimages.py 110 __init__() - No information on the spectrograph was given.  Do not attempt to (re)process the images
[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


In [14]:
# Maskslits
maskslits = np.array([False])

In [15]:
# datasec image
reload(arprocimg)
datasec_img, _, _ = arprocimg.get_datasec_trimmed(
    spectrograph, None, settings['detector']['num'],
    settings['detector'],
    naxis0=settings['detector']['naxis0'],
    naxis1=settings['detector']['naxis1'])
datasec_img.shape

(2048, 350)

In [16]:
# BPM
bpm = np.zeros_like(datasec_img)

## Instantiate

In [17]:
files = [pypdev_path+'/RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz']

In [18]:
reload(processimages)
reload(scienceimage)

sciI = scienceimage.ScienceImage(file_list=files,
                                    spectrograph=spectrograph,
                                     settings=settings,
                                     tilts=tilts,
                                     det=det,
                                     tslits_dict=traceSlits.tslits_dict,
                                     pixlocn=traceSlits.pixlocn,
                                     datasec_img=datasec_img,
                                     maskslits=maskslits,
                                     setup=setup,
                                         bpm=bpm
                                    )

## Process

In [19]:
_ = sciI._process(msbias, msflat, apply_gain=True, dnoise=0.)

[INFO]    :: arload.py 327 load_raw_frame() - Loading raw_file: /home/xavier/local/Python/PYPIT-development-suite//RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz
[INFO]    :: processimages.py 271 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: arprocimg.py 77 bias_subtract() - Subtracting bias image from raw frame
[WARNING] :: arprocimg.py 505 gain_frame() - Should probably be measuring the gain across the amplifier boundary
[INFO]    :: processimages.py 394 build_rawvarframe() - Generate raw variance frame (from detected counts [flat fielded])
[INFO]    :: arprocimg.py 236 lacosmic() - Detecting cosmic rays with the L.A.Cosmic algorithm
[WORK IN ]::
[PROGRESS]:: arprocimg.py 237 lacosmic() - Include these parameters in the settings files to be adjusted by the user
[INFO]    :: arprocimg.py 259 lacosmic() - Convolving image with Laplacian kernel
[INFO]    :: arprocimg.py 267 lacosmic() - Creating noise model
[INFO]    :: arprocimg.py 274 lacosmic() - Calculating Laplac

### View

In [20]:
sciI.show('sci')

In [21]:
sciI.show('rawvar')

In [22]:
sciI.show('crmasked')

## SkySub

In [23]:
settings['skysub'] = {}
settings['skysub']['perform']=True
settings['skysub']['method']= 'bspline'
settings['skysub']['bspline'] = {}
settings['skysub']['bspline']['everyn']=20
#
global_sky, modelvar = sciI.global_skysub(settings)

[INFO]    :: scienceimage.py 455 global_skysub() - Working on slit: 0
[INFO]    :: arskysub.py 38 bg_subtraction_slit() - Identifying pixels within each order
[INFO]    :: arskysub.py 42 bg_subtraction_slit() - Applying bad pixel mask
[INFO]    :: arskysub.py 60 bg_subtraction_slit() - Identifying pixels containing the science target
[WORK IN ]::
[PROGRESS]:: arskysub.py 61 bg_subtraction_slit() - Speed up this step with multi-processing
[INFO]    :: arskysub.py 130 bg_subtraction_slit() - Fitting sky background spectrum
[INFO]    :: arskysub.py 132 bg_subtraction_slit() - Using bspline sky subtraction
[INFO]    :: scienceimage.py 472 global_skysub() - Building model variance from the Sky frame


In [24]:
sciI.show('global')

In [25]:
sciI.show('image', image=sciI.sciframe-sciI.global_sky)

## Find Object

In [26]:
settings['trace']

{'object': {'find': 'standard',
  'function': 'legendre',
  'nsmooth': 3,
  'order': 2,
  'xedge': 0.03},
 'slits': {'diffpolyorder': 2,
  'fracignore': 0.01,
  'function': 'legendre',
  'maxgap': None,
  'maxshift': 0.15,
  'medrep': 0,
  'number': -1,
  'pad': 0.0,
  'pca': {'extrapolate': {'neg': 0, 'pos': 0},
   'params': [3, 2, 1, 0, 0, 0],
   'type': 'pixel'},
  'polyorder': 3,
  'sigdetect': 20.0,
  'single': [],
  'sobel': {'mode': 'nearest'}}}

In [27]:
tracelist, nobj = sciI.find_objects()
nobj

[INFO]    :: artrace.py 285 trace_objects_in_slit() - Rectifying science frame
[INFO]    :: artrace.py 306 trace_objects_in_slit() - Estimating object profiles
[INFO]    :: artrace.py 375 trace_objects_in_slit() - Identifying objects that are significantly detected
[INFO]    :: artrace.py 458 trace_objects_in_slit() - Found 1 object
[INFO]    :: artrace.py 459 trace_objects_in_slit() - Tracing 1 object
[INFO]    :: artrace.py 498 trace_objects_in_slit() - Performing global trace to all objects
[INFO]    :: artrace.py 501 trace_objects_in_slit() - Constructing a trace for all objects
[INFO]    :: artrace.py 507 trace_objects_in_slit() - Converting object trace to detector pixels
[INFO]    :: artrace.py 90 trace_objbg_image() - Creating an image weighted by object pixels for 1 objects
[INFO]    :: artrace.py 99 trace_objbg_image() - Creating an image weighted by background pixels


1

In [28]:
tracelist[0].keys()

dict_keys(['nobj', 'traces', 'object', 'params', 'background'])

## Another round of skysub

In [29]:
_ = sciI.global_skysub(settings, use_tracemask=True)

[INFO]    :: scienceimage.py 455 global_skysub() - Working on slit: 0
[INFO]    :: arskysub.py 38 bg_subtraction_slit() - Identifying pixels within each order
[INFO]    :: arskysub.py 42 bg_subtraction_slit() - Applying bad pixel mask
[INFO]    :: arskysub.py 60 bg_subtraction_slit() - Identifying pixels containing the science target
[WORK IN ]::
[PROGRESS]:: arskysub.py 61 bg_subtraction_slit() - Speed up this step with multi-processing
[INFO]    :: arskysub.py 130 bg_subtraction_slit() - Fitting sky background spectrum
[INFO]    :: arskysub.py 132 bg_subtraction_slit() - Using bspline sky subtraction
[INFO]    :: scienceimage.py 472 global_skysub() - Building model variance from the Sky frame


In [30]:
sciI.show('global')

In [31]:
sciI.show('skysub')

## Another round of finding objects

In [32]:
_, nobj = sciI.find_objects()
nobj

[INFO]    :: artrace.py 285 trace_objects_in_slit() - Rectifying science frame
[INFO]    :: artrace.py 306 trace_objects_in_slit() - Estimating object profiles
[INFO]    :: artrace.py 375 trace_objects_in_slit() - Identifying objects that are significantly detected
[INFO]    :: artrace.py 458 trace_objects_in_slit() - Found 1 object
[INFO]    :: artrace.py 459 trace_objects_in_slit() - Tracing 1 object
[INFO]    :: artrace.py 498 trace_objects_in_slit() - Performing global trace to all objects
[INFO]    :: artrace.py 501 trace_objects_in_slit() - Constructing a trace for all objects
[INFO]    :: artrace.py 507 trace_objects_in_slit() - Converting object trace to detector pixels
[INFO]    :: artrace.py 90 trace_objbg_image() - Creating an image weighted by object pixels for 1 objects
[INFO]    :: artrace.py 99 trace_objbg_image() - Creating an image weighted by background pixels


1

## Extraction

In [33]:
sciI.extraction(wave)

[INFO]    :: scienceimage.py 278 boxcar() - Performing boxcar extraction
[INFO]    :: arextract.py 80 boxcar() - Performing boxcar extraction of object 1/1 in slit 1/1
[INFO]    :: arextract.py 89 boxcar() -    Fitting the background
[INFO]    :: arextract.py 166 boxcar() -    Summing object counts
[INFO]    :: arextract.py 169 boxcar() -    Summing variance array
[INFO]    :: scienceimage.py 306 original_optimal() - Attempting optimal extraction with model profile
[WORK IN ]::
[PROGRESS]:: arextract.py 259 obj_profiles() - Should probably loop on S/N
[INFO]    :: arextract.py 261 obj_profiles() - Deriving spatial profile of object 1/1 in slit 1/1
[WORK IN ]::
[PROGRESS]:: artrace.py 1819 slit_image() - Use 2D spline to evaluate tilts
[WORK IN ]::
[PROGRESS]:: artrace.py 1824 slit_image() - Should worry about changing plate scale
[INFO]    :: arextract.py 284 obj_profiles() - Good S/N for profile
[WORK IN ]::
[PROGRESS]:: arextract.py 294 obj_profiles() - Weight by S/N in boxcar extrac

([[<SpecObjExp: O500-S5009-D01-I0000 == Setup S0-D0-G0-T0-B11 Object at 0.500431 in Slit at 0.500928 with det=01, scidx=0 and objtype=science>]],
 array([[ 19.35094331,  19.35094331,  19.35094331, ...,  19.35094331,
          19.35094331,  19.35094331],
        [ 19.35094331,  19.35094331,  19.35094331, ...,  19.35094331,
          19.35094331,  19.35094331],
        [ 19.35094331,  19.35094331,  19.35094331, ...,  19.35094331,
          19.35094331,  19.35094331],
        ..., 
        [ 19.35094331,  19.35094331,  19.35094331, ...,  19.35094331,
          19.35094331,  19.35094331],
        [ 19.35094331,  19.35094331,  19.35094331, ...,  19.35094331,
          19.35094331,  19.35094331],
        [ 19.35094331,  19.35094331,  19.35094331, ...,  19.35094331,
          19.35094331,  19.35094331]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0